<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250625/%E8%BC%B8%E9%80%81%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 輸送問題 演習問題 解説と解答

## 環境準備


In [1]:
# python-mip ライブラリをインストールする (セルの出力を非表示にする)
%%capture
!pip install mip

---
## 演習問題1

### 1. 問題
例題において、工場 $F_{1}$ から倉庫 $W_{2}$ への輸送コストが6から1に変化した場合、最小総輸送コストを求めよ。

### 2. 数理モデルの定式化
本問題は、例題で扱ったバランス型輸送問題が基礎となる。変更点は、目的関数における輸送コストの係数 $c_{F1,W2}$ が $6$ から $1$ に変更される点のみである。

* **インデックス**
  * $i$: 供給地(工場)のインデックス, $i \in \{F1, F2\}$
  * $j$: 需要地(倉庫)のインデックス, $j \in \{W1, W2, W3\}$

* **パラメータ**
  * 供給量 $s_i$: $s_{F1}=100, s_{F2}=180$
  * 需要量 $d_j$: $d_{W1}=80, d_{W2}=90, d_{W3}=110$
  * 輸送コスト $c_{ij}$:
$$
        C = \begin{pmatrix}
        4 & \mathbf{1} & 5 \\
        7 & 5 & 8
        \end{pmatrix}
$$

* **決定変数**
    * $x_{ij}$: 工場 $i$ から倉庫 $j$ への輸送量 (非負連続変数)

* **目的関数 (総輸送コスト最小化)**
    $$
    \min Z = 4x_{F1,W1} + \mathbf{1}x_{F1,W2} + 5x_{F1,W3} + 7x_{F2,W1} + 5x_{F2,W2} + 8x_{F2,W3}
    $$

* **制約条件**
    * 供給制約 (各工場の総出荷量は供給量に等しい):
$$
        \begin{aligned}
        x_{F1,W1} + x_{F1,W2} + x_{F1,W3} &= 100 \\
        x_{F2,W1} + x_{F2,W2} + x_{F2,W3} &= 180
        \end{aligned}
$$
    * 需要制約 (各倉庫の総入荷量は需要量に等しい):
        $$
        \begin{aligned}
        x_{F1,W1} + x_{F2,W1} &= 80 \\
        x_{F1,W2} + x_{F2,W2} &= 90 \\
        x_{F1,W3} + x_{F2,W3} &= 110
        \end{aligned}
        $$
    * 非負制約: $x_{ij} \ge 0$ for all $i, j$

### 3. Python (MIP) による実装
コストデータを変更し、再度最適化を行う。

In [2]:
import mip
import pandas as pd

# --- データ定義 ---
# 供給地と需要地
factories = ['F1', 'F2']
warehouses = ['W1', 'W2', 'W3']

# 供給量
supply = {'F1': 100, 'F2': 180}

# 需要量
demand = {'W1': 80, 'W2': 90, 'W3': 110}

# 輸送コスト (F1->W2 のコストを 1 に変更)
cost_data = [
    [4, 1, 5],  # F1からW1, W2, W3へのコスト
    [7, 5, 8]   # F2からW1, W2, W3へのコスト
]
costs = pd.DataFrame(cost_data, index=factories, columns=warehouses)

# --- モデル構築 ---
# モデルの作成 (最小化問題)
model = mip.Model(name="Transportation_ex1", sense=mip.MINIMIZE)

# 変数の定義 (輸送量 x[i, j])
x = {}
for i in factories:
    for j in warehouses:
        # 非負の連続変数を定義
        x[i, j] = model.add_var(name=f"x({i},{j})", var_type=mip.CONTINUOUS, lb=0)

# 制約条件の追加
# 供給制約: 各工場からの総出荷量は供給量に等しい
for i in factories:
    model.add_constr(mip.xsum(x[i, j] for j in warehouses) == supply[i], name=f"supply_{i}")

# 需要制約: 各倉庫への総入荷量は需要量に等しい
for j in warehouses:
    model.add_constr(mip.xsum(x[i, j] for i in factories) == demand[j], name=f"demand_{j}")

# 目的関数の設定 (総輸送コスト)
model.objective = mip.xsum(costs.loc[i, j] * x[i, j] for i in factories for j in warehouses)

# --- モデルの求解 ---
print("--- 求解開始 ---")
status = model.optimize()
print(f"求解ステータス: {status}")

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"\n最適解が発見されました。")
    print(f"最小総輸送コスト (最適値): {model.objective_value:.2f}")

    # 最適輸送計画をDataFrameで作成
    transport_plan = pd.DataFrame(index=factories, columns=warehouses)
    for i in factories:
        for j in warehouses:
            # 非常に小さい値は0として表示
            transport_plan.loc[i, j] = x[i, j].x if x[i, j].x > 1e-6 else 0

    print("\n--- 最適輸送計画 ---")
    print(transport_plan)
else:
    print("\n最適解が見つかりませんでした。")

--- 求解開始 ---
求解ステータス: OptimizationStatus.OPTIMAL

最適解が発見されました。
最小総輸送コスト (最適値): 1500.00

--- 最適輸送計画 ---
      W1    W2     W3
F1  10.0  90.0      0
F2  70.0     0  110.0


---
## 演習問題2

### 1. 問題
例題の状況で、倉庫 $W_{3}$ の需要量が110から130に増加した。総供給量が不足するため、この需要を満たすことはできない。この状況で、「可能な限り需要を満たし、かつ総輸送コストを最小化する」という目標を達成したい。ただし、満たせなかった需要に対するペナルティはないものとする。どのようなモデルを構築し、解けばよいか。その結果、最小輸送コストを求めよ。

### 2. 数理モデルの定式化
総供給量 ($100+180=280$) が総需要量 ($80+90+130=300$) を下回る、アンバランス型の輸送問題である。需要を完全に満たすことは不可能なため、需要制約を「需要量以下」の不等式に変更する。これにより、供給可能な総量 (280単位) の範囲内で、輸送コストが最小となるように各倉庫へ製品が分配される。

* **パラメータ**
  * 供給量 $s_i$: $s_{F1}=100, s_{F2}=180$
  * 需要量 $d_j$: $d_{W1}=80, d_{W2}=90, d_{W3}=\mathbf{130}$
  * 輸送コスト $c_{ij}$: (例題の元の値)
$$
        C = \begin{pmatrix}
        4 & 6 & 5 \\
        7 & 5 & 8
        \end{pmatrix}
$$

* **決定変数**:
    * $x_{ij}$: 工場 $i$ から倉庫 $j$ への輸送量 (非負連続変数)

* **目的関数 (総輸送コスト最小化)**:
    $$
    \min Z = 4x_{F1,W1} + 6x_{F1,W2} + 5x_{F1,W3} + 7x_{F2,W1} + 5x_{F2,W2} + 8x_{F2,W3}
    $$

* **制約条件**
  * 供給制約 (各工場の総出荷量は供給量でなければならない):
$$
        \begin{aligned}
        x_{F1,W1} + x_{F1,W2} + x_{F1,W3} & =  100 \\
        x_{F2,W1} + x_{F2,W2} + x_{F2,W3} & =  180
        \end{aligned}
$$
    * 需要制約 (各倉庫の総入荷量は需要量**以下**でも構わない):
$$
        \begin{aligned}
        x_{F1,W1} + x_{F2,W1} &\le 80 \\
        x_{F1,W2} + x_{F2,W2} &\le 90 \\
        x_{F1,W3} + x_{F2,W3} &\le 130
        \end{aligned}
$$
    * 非負制約: $x_{ij} \ge 0$ for all $i, j$


### 3. Python (MIP) による実装
需要量 `demand` の不等号を変更して実装する。

In [3]:
import mip
import pandas as pd

# --- データ定義 ---
# 供給地と需要地
factories = ['F1', 'F2']
warehouses = ['W1', 'W2', 'W3']

# 供給量
supply = {'F1': 100, 'F2': 180}

# 需要量 (W3の需要を130に変更)
demand = {'W1': 80, 'W2': 90, 'W3': 130}

# 輸送コスト (例題の元の値)
cost_data = [
    [4, 6, 5],
    [7, 5, 8]
]
costs = pd.DataFrame(cost_data, index=factories, columns=warehouses)

# --- モデル構築 ---
model = mip.Model(name="Transportation_ex2", sense=mip.MINIMIZE)

# 変数の定義
x = {}
for i in factories:
    for j in warehouses:
        x[i, j] = model.add_var(name=f"x({i},{j})", var_type=mip.CONTINUOUS, lb=0)

# 制約条件の追加
# 供給制約: 各工場からの総出荷量は供給量
for i in factories:
    model.add_constr(mip.xsum(x[i, j] for j in warehouses) == supply[i], name=f"supply_{i}")

# 需要制約: 各倉庫への総入荷量は需要量以下 (アンバランス型のため)
for j in warehouses:
    model.add_constr(mip.xsum(x[i, j] for i in factories) <= demand[j], name=f"demand_{j}")

# 目的関数の設定
model.objective = mip.xsum(costs.loc[i, j] * x[i, j] for i in factories for j in warehouses)

# --- モデルの求解 ---
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最適解が発見されました。")
    print(f"最小輸送コスト: {model.objective_value:.2f}")

    transport_plan = pd.DataFrame(index=factories, columns=warehouses, dtype=float)
    for i in factories:
        for j in warehouses:
            transport_plan.loc[i, j] = x[i, j].x if x[i, j].x > 1e-6 else 0

    print("\n--- 最適輸送計画 ---")
    print(transport_plan)

    # 満たされなかった需要の計算
    print("\n--- 需要充足状況 ---")
    total_transported = 0
    for j in warehouses:
        transported_to_j = transport_plan.sum(axis=0)[j]
        unmet_demand = demand[j] - transported_to_j
        total_transported += transported_to_j
        print(f"倉庫 {j}: 需要 {demand[j]}, 供給 {transported_to_j:.0f}, 不足 {unmet_demand:.0f}")

    print(f"\n総供給量: {sum(supply.values())}, 総輸送量: {total_transported:.0f}")

else:
    print("\n最適解が見つかりませんでした。")

最適解が発見されました。
最小輸送コスト: 1590.00

--- 最適輸送計画 ---
      W1    W2     W3
F1   0.0   0.0  100.0
F2  80.0  90.0   10.0

--- 需要充足状況 ---
倉庫 W1: 需要 80, 供給 80, 不足 0
倉庫 W2: 需要 90, 供給 90, 不足 0
倉庫 W3: 需要 130, 供給 110, 不足 20

総供給量: 280, 総輸送量: 280


---
## 演習問題3

### 1. 問題
例題の状況(コスト、供給量、需要量は元のバランス型に戻す)において、工場 $F_{2}$ から倉庫 $W_{2}$ への輸送路が工事のため、最大でも50単位しか輸送できないという制約が加わった。このときの最小総輸送コストを求めよ。

### 2. 数理モデルの定式化
元のバランス型輸送問題モデルに、特定の輸送経路に対する**上限制約**を追加する。

* **パラメータ**: 例題と同じ
* **決定変数**: 例題と同じ
* **目的関数**: 例題と同じ
* **制約条件**: 例題の供給制約、需要制約、非負制約に加えて、以下の制約を追加する。
    * 輸送量上限制約:
        $$
        x_{F2,W2} \le 50
        $$

### 3. Python (MIP) による実装
`model.add_var()` で変数を定義する際に `ub` (upper bound) パラメータを指定するか、あるいは `model.add_constr()` で明示的に制約を追加することでモデル化できる。ここでは後者の方法で実装する。

In [4]:
import mip
import pandas as pd

# --- データ定義 (例題の元の値) ---
factories = ['F1', 'F2']
warehouses = ['W1', 'W2', 'W3']
supply = {'F1': 100, 'F2': 180}
demand = {'W1': 80, 'W2': 90, 'W3': 110}
cost_data = [
    [4, 6, 5],
    [7, 5, 8]
]
costs = pd.DataFrame(cost_data, index=factories, columns=warehouses)

# --- モデル構築 ---
model = mip.Model(name="Transportation_ex3", sense=mip.MINIMIZE)

# 変数の定義
x = {}
for i in factories:
    for j in warehouses:
        x[i, j] = model.add_var(name=f"x({i},{j})", var_type=mip.CONTINUOUS, lb=0)

# 制約条件の追加
# 供給制約
for i in factories:
    model.add_constr(mip.xsum(x[i, j] for j in warehouses) == supply[i])

# 需要制約
for j in warehouses:
    model.add_constr(mip.xsum(x[i, j] for i in factories) == demand[j])

# ★★★ 追加の制約 ★★★
# F2からW2への輸送量の上限制約
model.add_constr(x['F2', 'W2'] <= 50, name="capacity_F2_W2")

# 目的関数の設定
model.objective = mip.xsum(costs.loc[i, j] * x[i, j] for i in factories for j in warehouses)

# --- モデルの求解 ---
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最適解が発見されました。")
    print(f"最小総輸送コスト: {model.objective_value:.2f}")

    transport_plan = pd.DataFrame(index=factories, columns=warehouses)
    for i in factories:
        for j in warehouses:
            transport_plan.loc[i, j] = x[i, j].x if x[i, j].x > 1e-6 else 0

    print("\n--- 最適輸送計画 ---")
    print(transport_plan)
else:
    print("\n最適解が見つかりませんでした。")

最適解が発見されました。
最小総輸送コスト: 1750.00

--- 最適輸送計画 ---
      W1    W2     W3
F1  60.0  40.0      0
F2  20.0  50.0  110.0


---
## 演習問題4

### 1. 問題
状況を以下のように変更する。
 * 供給量: $F_1: 130, F_2: 190$
 * 需要量: $W_1: 80, W_2: 60, W_3: 140$
 * コスト: 例題と同じ

輸送は特殊なコンテナ (20個単位でしか輸送できない)で行う必要がある場合、最小総輸送コストを解答せよ。

### 2. 数理モデルの定式化
輸送量が特定の数（20単位、ロットサイズと呼ばれる）
の整数倍でなければならないという制約を持つ。これは**整数計画問題 (Integer Programming, IP)** となる。
この制約をモデル化するため、輸送する「コンテナの数」を新しい決定変数として導入する。

* **インデックスとパラメータ**:
    * 供給量 $s_i$: $s_{F1}=130, s_{F2}=190$
    * 需要量 $d_j$: $d_{W1}=80, d_{W2}=60, d_{W3}=140$
    * コスト $c_{ij}$: 例題と同じ
    * ロットサイズ $L = 20$

* **決定変数**:
    * $y_{ij}$: 工場 $i$ から倉庫 $j$ へ輸送するコンテナの数 (**非負整数変数**)
    * 実際の輸送量は、 $20 \times y_{ij}$ となる。

* **目的関数 (総輸送コスト最小化)**:
    $$
    \min Z = \sum_{i \in \{F1,F2\}} \sum_{j \in \{W1,W2,W3\}} c_{ij} (20 \times y_{ij})
    $$

* **制約条件**:
    * 総供給量(320)が総需要量(280)を上回っているため、アンバランス型である。
    * 供給制約 (各工場の総出荷量は供給量以下):
$$
        \sum_{j \in \{W1,W2,W3\}} 20 \times y_{ij} \le s_i \quad \forall i \in \{F1,F2\}
$$
    * 需要制約 (各倉庫の総入荷量は需要量以上):
$$
        \sum_{i \in \{F1,F2\}} 20 \times y_{ij} \ge d_j \quad \forall j \in \{W1,W2,W3\}
$$
    * 非負整数制約: $y_{ij} \ge 0$ かつ整数 $\forall i, j$

### 3. Python (MIP) による実装
決定変数をコンテナ数（整数）として定義し、制約条件と目的関数をこの変数を用いて記述する。

In [5]:
import mip
import pandas as pd

# --- データ定義 ---
factories = ['F1', 'F2']
warehouses = ['W1', 'W2', 'W3']
supply = {'F1': 130, 'F2': 190}
demand = {'W1': 80, 'W2': 60, 'W3': 140}
cost_data = [
    [4, 6, 5],
    [7, 5, 8]
]
costs = pd.DataFrame(cost_data, index=factories, columns=warehouses)
lot_size = 20

# --- モデル構築 ---
model = mip.Model(name="Transportation_ex4_Integer", sense=mip.MINIMIZE)

# 変数の定義 (y[i,j]はコンテナの数)
y = {}
for i in factories:
    for j in warehouses:
        # 非負の「整数」変数を定義
        y[i, j] = model.add_var(name=f"y({i},{j})", var_type=mip.INTEGER, lb=0)

# 制約条件の追加
# 供給制約: 実際の輸送量(ロットサイズ*コンテナ数)が供給量以下
for i in factories:
    model.add_constr(mip.xsum(lot_size * y[i, j] for j in warehouses) <= supply[i])

# 需要制約: 実際の輸送量が需要量以上
for j in warehouses:
    model.add_constr(mip.xsum(lot_size * y[i, j] for i in factories) >= demand[j])

# 目的関数の設定 (コスト * 実際の輸送量)
model.objective = mip.xsum(costs.loc[i, j] * (lot_size * y[i, j]) for i in factories for j in warehouses)

# --- モデルの求解 ---
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最適解が発見されました。")
    print(f"最小総輸送コスト: {model.objective_value:.2f}")

    # 実際の輸送量 (単位) を計算
    transport_units = pd.DataFrame(index=factories, columns=warehouses)
    for i in factories:
        for j in warehouses:
            transport_units.loc[i, j] = y[i, j].x * lot_size if y[i, j].x > 1e-6 else 0

    print("\n--- 最適輸送計画 (単位) ---")
    print(transport_units)

    # コンテナ数も表示
    transport_containers = pd.DataFrame(index=factories, columns=warehouses)
    for i in factories:
        for j in warehouses:
             transport_containers.loc[i, j] = y[i, j].x if y[i, j].x > 1e-6 else 0

    print("\n--- 最適輸送計画 (コンテナ数) ---")
    print(transport_containers)

else:
    print("\n最適解が見つかりませんでした。")

最適解が発見されました。
最小総輸送コスト: 1620.00

--- 最適輸送計画 (単位) ---
      W1    W2     W3
F1  80.0     0   40.0
F2     0  60.0  100.0

--- 最適輸送計画 (コンテナ数) ---
     W1   W2   W3
F1  4.0    0  2.0
F2    0  3.0  5.0


---
## 演習問題5

### 1. 問題
2つの工場が2種類の商品(A, B)を生産し、3つの倉庫に輸送する。データは以下の通り。
* 商品別供給量: $F_1(A:100, B:80), F_2(A:100, B:80)$
* 商品別需要量: $W_1(A:40, B:30), W_2(A:30, B:40), W_3(A:50, B:30)$
* 輸送コスト: 商品によらず経路のみで決まる (例題と同じ)
* 追加制約（工場の総出荷能力）:
    * $F_1$: 商品A, B合わせて最大140単位
    * $F_2$: 商品A, B合わせて最大90単位

総輸送コストを最小にする輸送計画と、そのコストを求めよ。

### 2. 数理モデルの定式化
この問題は、複数の商品を同時に扱う**多品種輸送問題**である。商品は区別されるため、決定変数に商品のインデックスを追加する必要がある。

* **インデックス**:
  * $i$: 工場, $i \in \{F1, F2\}$
  * $j$: 倉庫, $j \in \{W1, W2, W3\}$
  * $k$: 商品, $k \in \{A, B\}$

* **パラメータ**:
    * 商品別供給量 $s_{ik}$
    * 商品別需要量 $d_{jk}$
    * 輸送コスト $c_{ij}$
    * 工場総出荷能力 $Cap_i$

* **決定変数**:
    * $x_{ijk}$: 工場 $i$ から倉庫 $j$ へ商品 $k$ を輸送する量 (非負連続変数)

* **目的関数 (総輸送コスト最小化)**:
    $$
    \min Z = \sum_{i, j, k} c_{ij} x_{ijk}
    $$

* **制約条件**:
    * 商品別供給制約: 各工場$i$からの各商品$k$の出荷量は、その商品の供給量以下。
        $$
        \sum_{j} x_{ijk} \le s_{ik} \quad \forall i, k
        $$
    * 商品別需要制約: 各倉庫$j$への各商品$k$の入荷量は、その商品の需要量以上。
        $$
        \sum_{i} x_{ijk} \ge d_{jk} \quad \forall j, k
        $$
    * 工場総出荷能力制約: 各工場$i$からの全商品の総出荷量は、その工場の能力以下。
        $$
        \sum_{j} \sum_{k} x_{ijk} \le Cap_i \quad \forall i
        $$
    * 非負制約: $x_{ijk} \ge 0 \quad \forall i, j, k$

### 3. Python (MIP) による実装
3次元のインデックスを持つ変数を定義し、上記の制約をすべてモデルに追加する。

In [6]:
import mip
import pandas as pd

# --- データ定義 ---
factories = ['F1', 'F2']
warehouses = ['W1', 'W2', 'W3']
products = ['A', 'B']

# 商品別供給量
supply_data = {'A': {'F1': 100, 'F2': 100}, 'B': {'F1': 80, 'F2': 80}}
supply = pd.DataFrame(supply_data)

# 商品別需要量
demand_data = {'A': {'W1': 40, 'W2': 30, 'W3': 50}, 'B': {'W1': 30, 'W2': 40, 'W3': 30}}
demand = pd.DataFrame(demand_data)

# 輸送コスト (商品によらない)
cost_data = [[4, 6, 5], [7, 5, 8]]
costs = pd.DataFrame(cost_data, index=factories, columns=warehouses)

# 工場総出荷能力
capacity = {'F1': 140, 'F2': 90}

# --- モデル構築 ---
model = mip.Model(name="Multi_Commodity_Transport", sense=mip.MINIMIZE)

# 変数の定義 (x[i,j,k])
x = {}
for i in factories:
    for j in warehouses:
        for k in products:
            x[i, j, k] = model.add_var(name=f"x({i},{j},{k})", lb=0)

# 制約条件の追加
# 1. 商品別供給制約
for i in factories:
    for k in products:
        model.add_constr(mip.xsum(x[i, j, k] for j in warehouses) <= supply.loc[i, k])

# 2. 商品別需要制約
for j in warehouses:
    for k in products:
        model.add_constr(mip.xsum(x[i, j, k] for i in factories) >= demand.loc[j, k])

# 3. 工場総出荷能力制約
for i in factories:
    model.add_constr(mip.xsum(x[i, j, k] for j in warehouses for k in products) <= capacity[i])

# 目的関数の設定
model.objective = mip.xsum(costs.loc[i, j] * x[i, j, k] for i in factories for j in warehouses for k in products)

# --- モデルの求解 ---
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最適解が発見されました。")
    print(f"最小総輸送コスト: {model.objective_value:.2f}")

    print("\n--- 最適輸送計画 ---")
    for k in products:
        print(f"\n--- 商品 {k} ---")
        plan = pd.DataFrame(index=factories, columns=warehouses)
        for i in factories:
            for j in warehouses:
                plan.loc[i, j] = x[i, j, k].x if x[i, j, k].x > 1e-6 else 0
        print(plan)
else:
    print("\n最適解が見つかりませんでした。")

最適解が発見されました。
最小総輸送コスト: 1060.00

--- 最適輸送計画 ---

--- 商品 A ---
      W1    W2    W3
F1  30.0     0  50.0
F2  10.0  30.0     0

--- 商品 B ---
      W1    W2    W3
F1  30.0     0  30.0
F2     0  40.0     0


---
## 演習問題6

### 1. 問題
例題の状況で、コストを以下のように変更する。
 * 工場 F1から倉庫 W3への輸送コストを7とする。
 * 工場 F1から倉庫 W1への輸送コストが、輸送量に応じて変化する（区分的線形コスト）。
     * 最初の30単位まで: 単位あたり4
     * 30単位を超え70単位まで: 超過分に単位あたり5
     * 70単位を超える分: 超過分に単位あたり6

最小総輸送コストを解答せよ。

### 2. 数理モデルの定式化
輸送コストが輸送量に応じて増加する（凸関数である）ため、この区分的線形コストは、輸送量変数を各区間に対応する複数の新しい変数に分割することで線形計画問題としてモデル化できる。

* 変数 $x_{F1,W1}$ を3つの変数に分割する:
    * $x_{seg1}$: 輸送量 0～30 の区間を担当 (コスト4)
    * $x_{seg2}$: 輸送量 30～70 の区間を担当 (コスト5)
    * $x_{seg3}$: 輸送量 70以上の区間を担当 (コスト6)

* **決定変数**:
    * $x_{ij}$: 工場 $i$ から倉庫 $j$ への輸送量。
    * $x_{seg1}, x_{seg2}, x_{seg3}$: F1からW1への輸送量を区間ごとに表す変数

* **目的関数**:
$$
    \min Z = \sum_{(i,j) \not = (F1,W1)} c_{ij} x_{ij}
    + 4 x_{seg1} + 5 x_{seg2}+ 6 x_{seg3}
$$

* **制約条件**:
    * 各分割変数には、その区間の幅に対応する上限制約を設ける。
$$
        \begin{aligned}
        0 \le x_{seg1} \le 30 \\
        0 \le x_{seg2} \le 40 \quad (= 70 - 30) \\
        x_{seg3} \ge 0
        \end{aligned}
$$
  
    * 元の制約に加えて、分割した変数の関係を定義する制約を追加する。
    すなわち、F1からW1への総輸送量は、分割された変数の合計である。
        $$
        x_{F1,W1} = x_{seg1} + x_{seg2} + x_{seg3}
        $$

    * 供給制約と需要制約は同じである。
    
### 3. Python (MIP) による実装

In [7]:
import mip
import pandas as pd

# --- データ定義 ---
factories = ['F1', 'F2']
warehouses = ['W1', 'W2', 'W3']
supply = {'F1': 100, 'F2': 180}
demand = {'W1': 80, 'W2': 90, 'W3': 110}

# 輸送コスト (F1->W3を7に変更、F1->W1は特別扱い)
# DataFrameからはF1->W1のコストは除外しておく
cost_data = {
    'W1': {'F1': float('nan'), 'F2': 7},
    'W2': {'F1': 6, 'F2': 5},
    'W3': {'F1': 7, 'F2': 8}
}
costs = pd.DataFrame(cost_data)

# --- モデル構築 ---
model = mip.Model(name="Transportation_ex6_Piecewise", sense=mip.MINIMIZE)

# 変数の定義
# 通常の経路の変数
x = {}
for i in factories:
    for j in warehouses:
        x[i, j] = model.add_var(name=f"x({i},{j})", lb=0)

# F1->W1経路の分割された変数
x_f1w1_seg1 = model.add_var(name="x_f1w1_seg1", lb=0, ub=30)
x_f1w1_seg2 = model.add_var(name="x_f1w1_seg2", lb=0, ub=40)
x_f1w1_seg3 = model.add_var(name="x_f1w1_seg3", lb=0)

# F1からW1への総輸送量
model.add_constr(x['F1','W1'] == x_f1w1_seg1 + x_f1w1_seg2 + x_f1w1_seg3)

# 制約条件の追加
# 供給制約
model.add_constr(x['F1', 'W1'] + x['F1', 'W2'] + x['F1', 'W3'] == supply['F1'])
model.add_constr(x['F2', 'W1'] + x['F2', 'W2'] + x['F2', 'W3'] == supply['F2'])

# 需要制約
model.add_constr(x['F1', 'W1'] + x['F2', 'W1'] == demand['W1'])
model.add_constr(x['F1', 'W2'] + x['F2', 'W2'] == demand['W2'])
model.add_constr(x['F1', 'W3'] + x['F2', 'W3'] == demand['W3'])

# 目的関数の設定
# 分割変数のコスト
objective_f1w1 = 4 * x_f1w1_seg1 + 5 * x_f1w1_seg2 + 6 * x_f1w1_seg3
# 通常変数のコスト
objective_others = mip.xsum(costs.loc[i, j] * x[i, j] for i in factories for j in warehouses if not (i == 'F1' and j == 'W1'))

model.objective = objective_f1w1 + objective_others

# --- モデルの求解 ---
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最適解が発見されました。")
    print(f"最小総輸送コスト: {model.objective_value:.2f}")

    # F1->W1 の輸送量を確認
    f1w1_val = x_f1w1_seg1.x + x_f1w1_seg2.x + x_f1w1_seg3.x
    print(f"\nF1 -> W1 への総輸送量: {f1w1_val:.2f}")
    print(f"  - 第1区間 (コスト4): {x_f1w1_seg1.x:.2f}")
    print(f"  - 第2区間 (コスト5): {x_f1w1_seg2.x:.2f}")
    print(f"  - 第3区間 (コスト6): {x_f1w1_seg3.x:.2f}")

    # 全体の輸送計画をDataFrameで表示
    plan = pd.DataFrame(index=factories, columns=warehouses, dtype=float)
    for i in factories:
        for j in warehouses:
            if i == 'F1' and j == 'W1':
                plan.loc[i, j] = f1w1_val
            else:
                plan.loc[i, j] = x[i, j].x if x[i, j].x > 1e-6 else 0
    print("\n--- 最適輸送計画 ---")
    print(plan)

else:
    print("\n最適解が見つかりませんでした。")

最適解が発見されました。
最小総輸送コスト: 1690.00

F1 -> W1 への総輸送量: 70.00
  - 第1区間 (コスト4): 30.00
  - 第2区間 (コスト5): 40.00
  - 第3区間 (コスト6): 0.00

--- 最適輸送計画 ---
      W1    W2    W3
F1  70.0   0.0  30.0
F2  10.0  90.0  80.0


In [8]:
import mip
import pandas as pd

# --- データ定義 ---
factories = ['F1', 'F2']
warehouses = ['W1', 'W2', 'W3']
supply = {'F1': 100, 'F2': 180}
demand = {'W1': 80, 'W2': 90, 'W3': 110}

# 輸送コスト (F1->W3を7に変更、F1->W1は特別扱い)
# DataFrameからはF1->W1のコストは除外しておく
cost_data = {
    'W1': {'F1': float('nan'), 'F2': 7},
    'W2': {'F1': 6, 'F2': 5},
    'W3': {'F1': 7, 'F2': 8}
}
costs = pd.DataFrame(cost_data)

# --- モデル構築 ---
model = mip.Model(name="Transportation_ex6_Piecewise", sense=mip.MINIMIZE)

# 変数の定義
# 通常の経路の変数
x = {}
for i in factories:
    for j in warehouses:
        # F1からW1への経路は除く
        if not (i == 'F1' and j == 'W1'):
            x[i, j] = model.add_var(name=f"x({i},{j})", lb=0)

# F1->W1経路の分割された変数
x_f1w1_seg1 = model.add_var(name="x_f1w1_seg1", lb=0, ub=30)
x_f1w1_seg2 = model.add_var(name="x_f1w1_seg2", lb=0, ub=40)
x_f1w1_seg3 = model.add_var(name="x_f1w1_seg3", lb=0)

# F1からW1への総輸送量
x_f1w1_total = x_f1w1_seg1 + x_f1w1_seg2 + x_f1w1_seg3

# 制約条件の追加
# 供給制約
# F1: 分割変数を使って表現
model.add_constr(x_f1w1_total + x['F1', 'W2'] + x['F1', 'W3'] == supply['F1'])
# F2: 通常通り
model.add_constr(x['F2', 'W1'] + x['F2', 'W2'] + x['F2', 'W3'] == supply['F2'])

# 需要制約
# W1: 分割変数を使って表現
model.add_constr(x_f1w1_total + x['F2', 'W1'] == demand['W1'])
# W2, W3: 通常通り
model.add_constr(x['F1', 'W2'] + x['F2', 'W2'] == demand['W2'])
model.add_constr(x['F1', 'W3'] + x['F2', 'W3'] == demand['W3'])

# 目的関数の設定
# 分割変数のコスト
objective_f1w1 = 4 * x_f1w1_seg1 + 5 * x_f1w1_seg2 + 6 * x_f1w1_seg3
# 通常変数のコスト
objective_others = mip.xsum(costs.loc[i, j] * x[i, j] for i in factories for j in warehouses if not (i == 'F1' and j == 'W1'))

model.objective = objective_f1w1 + objective_others

# --- モデルの求解 ---
status = model.optimize()

# --- 結果の表示 ---
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最適解が発見されました。")
    print(f"最小総輸送コスト: {model.objective_value:.2f}")

    # F1->W1 の輸送量を確認
    f1w1_val = x_f1w1_seg1.x + x_f1w1_seg2.x + x_f1w1_seg3.x
    print(f"\nF1 -> W1 への総輸送量: {f1w1_val:.2f}")
    print(f"  - 第1区間 (コスト4): {x_f1w1_seg1.x:.2f}")
    print(f"  - 第2区間 (コスト5): {x_f1w1_seg2.x:.2f}")
    print(f"  - 第3区間 (コスト6): {x_f1w1_seg3.x:.2f}")

    # 全体の輸送計画をDataFrameで表示
    plan = pd.DataFrame(index=factories, columns=warehouses, dtype=float)
    for i in factories:
        for j in warehouses:
            if i == 'F1' and j == 'W1':
                plan.loc[i, j] = f1w1_val
            else:
                plan.loc[i, j] = x[i, j].x if x[i, j].x > 1e-6 else 0
    print("\n--- 最適輸送計画 ---")
    print(plan)

else:
    print("\n最適解が見つかりませんでした。")

最適解が発見されました。
最小総輸送コスト: 1690.00

F1 -> W1 への総輸送量: 70.00
  - 第1区間 (コスト4): 30.00
  - 第2区間 (コスト5): 40.00
  - 第3区間 (コスト6): 0.00

--- 最適輸送計画 ---
      W1    W2    W3
F1  70.0   0.0  30.0
F2  10.0  90.0  80.0
